In [ ]:
from __future__ import print_function

import google.auth
import json
from httplib2 import Http
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Connect to Google API

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive"]

"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


In [ ]:
try:
    # service = build('drive', 'v3', credentials=creds)
    service = build('docs', 'v1', credentials=creds)
except HttpError as error:
    print(f'An error occurred: {error}')

# Get Document by Id

In [ ]:
document = service.documents().get(documentId="1TRYIqpFZ7zK6tLoPAp3nKl_-lcja7E9gJdRtdJzCTDo").execute()
print(json.dumps(document, indent=4, sort_keys=True))

# List Files in Folder

# List Subfolders in Folder

In [ ]:
service = build('drive', 'v3', credentials=creds)

In [ ]:
# https://developers.google.com/drive/api/guides/search-files
# Call the Drive v3 API

results = service.files().list(
    pageSize=200,
    q="'14YftDicRvNzo4uUUsL0ubG7oB8Ayu6AJ' in parents and mimeType = 'application/vnd.google-apps.folder'",
    fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

for item in items:
    # print(u'{0} ({1})'.format(item['name'], item['id']))    
    # print(f"=HYPERLINK(\"https://drive.google.com/drive/folders/{item['id']}\",\"{item['name']}\")")
    # print(f"{item['name']}")
    print(f"https://drive.google.com/drive/folders/{item['id']}")



In [ ]:
# Call the Drive v3 API
parent_folder = "1-dau0E71lAZjBbT9789Mqk9eCp_tvvJw"
results = service.files().list(
    pageSize=100,
    q=f"'{parent_folder}' in parents",
    fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

print('Files:')
for item in items:
    print(f"=HYPERLINK(\"https://drive.google.com/drive/folders/{item['id']}\",\"{item['name']}\")")

In [ ]:
def create_short_cut(name, id, folder_id):
    shortcut_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {
            'targetId': id
        },
        'parents': [folder_id]
    }
    return service.files().create(body=shortcut_metadata, fields='id,shortcutDetails').execute()


In [ ]:
# https://developers.google.com/drive/api/guides/search-files
service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=20,
    q="'1ejkPzITGEurNimytA0Xyk4e9FKVNLDSy' in parents and mimeType = 'application/vnd.google-apps.folder'",
    fields="nextPageToken, files(id, name)").execute()
letter_folders = list(results.get('files', []))

folder_names = ["Affaires climatiques", "Affaires environnementales"]
env_folder_target_id = "1pxBuo72r20y16zXWIH31nJuNbfqVfku6"
cli_folder_target_id = "1-dau0E71lAZjBbT9789Mqk9eCp_tvvJw"

for letter_folder in letter_folders:
    print(letter_folder['name'])

    # Affaires climatiques
    climate_res = service.files().list(
        pageSize=10,
        q=f"'{letter_folder['id']}' in parents and mimeType = 'application/vnd.google-apps.folder' and name = 'Affaires climatiques'",
        fields="nextPageToken, files(id, name)").execute()
    climate_folder = climate_res.get('files', [])
    for folder in climate_folder:
        print(folder["name"])
        
        # copy in
        files = service.files().list(
            pageSize=20,
            q=f"'{folder['id']}' in parents and mimeType = 'application/vnd.google-apps.shortcut'",
            fields="nextPageToken, files(id, name)").execute().get('files', [])
        for file in files:
            print("Affaires climatiques", file["name"])
            file_metadata = {
                'parents': [cli_folder_target_id],
                'name': file["name"]
            }
            service.files().copy(fileId=file["id"], body=file_metadata).execute()

        # make shortcut in
        files = service.files().list(
            pageSize=20,
            q=f"'{folder['id']}' in parents and mimeType != 'application/vnd.google-apps.shortcut'",
            fields="nextPageToken, files(id, name)").execute().get('files', [])
        for file in files:
            print("Affaires climatiques", file["name"])
            create_short_cut(name=file["name"], id=file["id"], folder_id=cli_folder_target_id)
    

    # Affaires environnementales
    env_res = service.files().list(
        pageSize=10,
        q=f"'{letter_folder['id']}' in parents and mimeType = 'application/vnd.google-apps.folder' and name = 'Affaires environnementales'",
        fields="nextPageToken, files(id, name)").execute()
    env_folder = env_res.get('files', [])
    for folder in env_folder:
        print(folder["name"])

        # copy in
        files = service.files().list(
            pageSize=20,
            q=f"'{folder['id']}' in parents and mimeType = 'application/vnd.google-apps.shortcut'",
            fields="nextPageToken, files(id, name)").execute().get('files', [])
        for file in files:
            print("Affaires environnementales", file["name"])
            file_metadata = {
                'parents': [env_folder_target_id],
                'name': file["name"]
            }
            service.files().copy(fileId=file["id"], body=file_metadata).execute()

        # make shortcut in
        files = service.files().list(
            pageSize=20,
            q=f"'{folder['id']}' in parents and mimeType != 'application/vnd.google-apps.shortcut'",
            fields="nextPageToken, files(id, name)").execute().get('files', [])
        for file in files:
            print("Affaires environnementales", file["name"])
            create_short_cut(name=file["name"], id=file["id"], folder_id=env_folder_target_id)

    

In [ ]:
def create_folder(name, folder_id):
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [folder_id]
    }
    return service.files().create(body=folder_metadata, fields='id').execute()

In [ ]:
# Affaires environnementales
parent_dir = '1-dau0E71lAZjBbT9789Mqk9eCp_tvvJw'
# 1-dau0E71lAZjBbT9789Mqk9eCp_tvvJw
# 1pxBuo72r20y16zXWIH31nJuNbfqVfku6

env_res = service.files().list(
    pageSize=50,
    q=f"'{parent_dir}' in parents and mimeType != 'application/vnd.google-apps.folder'",
    fields="nextPageToken, files(id, name)").execute()
env_files = env_res.get('files', [])
for env_file in env_files:
    print(env_file["name"])

    folder = create_folder(env_file["name"].replace(".docx", "").replace(".pdf", ""), parent_dir)

    file_metadata = {
        'parents': [folder.get('id')],
        'name': env_file["name"]
    }
    service.files().copy(fileId=env_file["id"], body=file_metadata).execute()
